In [5]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
from tesserocr import PyTessBaseAPI
import os
from PIL import Image
import json

In [6]:
#loading the template

template = json.loads(open(r"C:\Users\brans\Downloads\template.json").read())
template = json.loads(template)

template_dimension = template.get("image_dimension")
template_data_fields = template.get("data_fields")


directory = r"C:\Users\brans\Downloads\data"
files = []
filenames = []
# iterate over files in that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        files.append(f)
        filenames.append(filename)

data = {}        

for h in range(len(files)):
    path = r"{}".format(files[h])
    
    #loading the image
    img =  cv2.imread(path)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    thresh_inv = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

    # Blur the image
    blur = cv2.GaussianBlur(thresh_inv,(1,1),0)

    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

    #resizing image to match the template
    thresh = cv2.resize(thresh, (template_dimension[0], template_dimension[1]))
    
    
    temp_data = {}

    for i in range(len(template_data_fields)):

            c1, c2, c3, c4 = template_data_fields[i][0][0], template_data_fields[i][0][1], template_data_fields[i][0][2], template_data_fields[i][0][3]
            image_arr = img
            image_arr = image_arr[c1:c2, c3:c4]

            # Temporary saving image for OCR processing
            im = Image.fromarray(image_arr)
            im.save(r"C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\temp\image_arr.jpeg")

            # OCR process
            with PyTessBaseAPI(path=r'C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\Tessdata', lang='eng_best') as api:
                api.SetImageFile(r"C:\Users\brans\OneDrive\Documents\Application-files-utilities\VScode\Project-007\temp\image_arr.jpeg")
                temp_data[template_data_fields[i][1][0]] = api.GetUTF8Text()
                
    data[filenames[h]] = temp_data
    
    
    





FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\brans\\Downloads\\data'

In [13]:
data_json = json.dumps(data)

with open(r"C:\Users\brans\Downloads\data.json", "w") as outfile:
    json.dump(data_json, outfile)

In [10]:
print(data)

{'filled-form-1.jpg': {'First\n\n \n': 'Aaron\n', '': ''}, 'filled-form-2.jpg': {'First\n\n \n': 'Aaron\n', '': ''}}
